In [77]:
import numpy as np
import math
import time

import pickle
import mne
import eegraph
print(mne.__version__)

import gradio as gr
print(gr.__version__) 

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

#import matplotlib
#matplotlib.use('Agg') #this will invoke matplotibs silent mode in which the figures are produced but not shown.

#import matplotlib.pyplot as plt
# add plot inline in the page
%matplotlib inline
mne.set_log_level('WARNING')

1.0.3
3.0.24


In [84]:
biosemi_montage_data = mne.channels.make_standard_montage('biosemi64') #'standard_1020'
biosemi_montage_epochs = mne.channels.make_standard_montage('biosemi64')

def freq_gen(file_obj):
    global G,graphs,connectivity_matrix,n_graphs
    raw_fname=file_obj.name
    data=mne.io.read_raw(raw_fname,exclude=('EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8','Status'))
    def events_n_epochs(raw_data):
        events = []
        sampling_freq = raw_data.info['sfreq'] #256
        np_data = raw_data.get_data()
        n_timepoints = np_data.shape[-1] #77568
        for i in range(n_timepoints//int(sampling_freq)):
            events.append([i*int(sampling_freq),0,1])
        events = np.array(events)
        epochs = mne.Epochs(raw_data, events)
        evoked = epochs.average()
        cov = mne.compute_covariance (epochs, tmax=0)
        return events,epochs
    
    events, epochs=events_n_epochs(data)
    data.set_montage(biosemi_montage_data) #epochs.plot_sensors(show_names=True)
    epochs.set_montage(biosemi_montage_epochs)
    
    
## DATA VISUALS

    #Sensor plot
    data.plot_sensors(show_names=True).savefig("plot_sensors.png")
    #epochs.plot_sensors(show_names=True)
    
    #Power Spectral Density/psd
    data.plot_psd(picks='eeg').savefig("plot_psd.png") ##visualize the frequency content of continuous data
    
    #psd topo
    data.pick_types(meg=False, eeg=True).plot_psd_topo().savefig("plot_psd_topo.png")
    
    #Evoked data Time-points topo
    data.set_eeg_reference(projection=True)
    cov = mne.compute_covariance(epochs, tmax=0.)
    evoked = epochs['1'].average()  # trigger 1 in auditory/left
    evoked.plot_joint().savefig("plot_evoked_topo.png")#standard sensor-space operations like make joint plots of evoked data.
    
    #Independent component analysis/ica
    # set up and fit the ICA
    ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter='auto')
    ica.fit(data)
    #ica.exclude = [1, 2]  # to display only selected ICAs
    ica_figure=ica.plot_properties(data) #picks=[0, 1] #(data, picks=ica.exclude)
    ica_figure[0].savefig("plot_ica.png")
    
## EPOCH VISUALS
    
    
    return 'Select an option from below',gr.update(visible=True)

def plot(option):
    if option== 'Sensor plot':
        image= 'plot_sensors.png'
    if option== 'Power Spectral Density/psd':
        image= 'plot_psd.png'
    if option== 'psd topo':
        image= 'plot_psd_topo.png'
    if option== 'Time-points topo of Evoked data':
        image= 'plot_evoked_topo.png'
    if option== 'Independent component analysis/ica':
        image= 'plot_ica.png'
    return image
        
    
    

#####################################################  


with gr.Blocks() as demo1:
    gr.Image('neuroinformatics_logo.png',image_mode='L',label='AI in Tinnitus').style(height=54, width=240)
    gr.Markdown(
    """
    # EEG data Visualisation
    Once the data is ready to visualise, the options should be visible.
    """
    )

    file = gr.File(label="Upload eeg data")    
    
    status=gr.Label(label='status')
    
    with gr.Row(visible=False) as outs1:
        data_btn = gr.Button("Visualise continous data",variant='primary')
        epoch_btn = gr.Button("Visualise epochs",variant='primary')
        
    with gr.Column(visible=False) as outs2:
        menu = gr.Radio(label="Visualise?", choices=["Sensor plot", "Power Spectral Density/psd","psd topo","Time-points topo of Evoked data","Independent component analysis/ica"])
        visual = gr.Image()
    with gr.Column(visible=False) as outs3:
        epoch_slider=gr.Slider(0,10,step=1,interactive=True)
    
    file.change(freq_gen,file,[status,outs1])
    
    
    def update1():
        return gr.update(visible=True),gr.update(visible=False)
    def update2():
        return gr.update(visible=True),gr.update(visible=False)

    data_btn.click(update1, None, [outs2,outs3])
    epoch_btn.click(update2, None, [outs3,outs2])

    menu.change(plot, menu, visual)
    #epoch_slider.change()


In [85]:
def graph_gen(file_obj,measure):
    global G,graphs,connectivity_matrix,n_graphs
    raw_fname=file_obj.name
    G = eegraph.Graph()
    G.load_data(path = raw_fname,electrode_montage_path = '1.ced')
    if measure=="pearson_correlation":   
        graphs, connectivity_matrix = G.modelate(window_size = 2, connectivity = 'pearson_correlation')#threshold = 0.8
    elif measure=="squared_coherence":
        graphs, connectivity_matrix = G.modelate(window_size = 2, connectivity = 'squared_coherence',bands = ['delta','theta','alpha'])
    n_graphs=len(graphs)
    framenum=len(connectivity_matrix)
    A1=[]
    for n in range(framenum):
        A=connectivity_matrix[n,:,:] 
        A=A.flatten()
        A1.append(A)
    A1 = np.array(A1) 
    with open('FunctionalConnectivitymodel.pickle','rb') as fr:
       graphclassificationmodel = pickle.load(fr)
       predicted_labels=graphclassificationmodel.predict(A1)   
    mask = np.unique(predicted_labels)
    tmp = []
    for v in mask:
       tmp.append(np.sum(predicted_labels==v))
       ts = np.max(tmp)
       Total=predicted_labels.size
       ts1=Total-ts
       max_v = mask[np.argmax(tmp)]
       Prob=ts/Total*100
       Prob1=ts1/Total*100
       if max_v==1:
           pred='Responder'
       else:
           pred='Non-Responder'
    return (f'{round(Prob,2)}% chance this person is {pred}'),(f'{n_graphs} connectivity graphs generated')

    
def visualise(number): 
    grph_number=number-1
    #G.visualize(graphs[grph_number], 'graph_1')
    fig=eegraph.draw_graph(graphs[grph_number])
    fig.update_layout(margin=dict(l=15,r=10),width=1400, height=700)
    #path='graph_1_plot.html'
    return fig

#####################################################    



with gr.Blocks() as demo2:
    gr.Image('neuroinformatics_logo.png',image_mode='L',label='AI in Tinnitus').style(height=54, width=240)
    gr.Markdown(
    """
    # Tinnitus Prediction using Functional Connectivity
    """)
    gr.Markdown(
    """
    Upload the eeg data to see the prediction.
    
    """)
    inp1 = gr.File(label="Upload eeg data")
    inp2 = gr.Radio(["pearson_correlation", "squared_coherence"],label="Select a connectivity measure",value="pearson_correlation")
    btn1 = gr.Button("Compute Functional Connectivity and Make the Prediction",variant='primary')
    out1 = gr.Label(label="Predicted Results")
    lab1 = gr.Label(label="Graphs Generated")
    btn1.click(fn=graph_gen, inputs=[inp1,inp2], outputs=[out1,lab1])
    gr.Markdown(
    """
  ***********************************************************************************************
  ***********************************************************************************************
  
    """)
    gr.Markdown(
    """
    # Visualise the Connectivity Graph
    """)
    gr.Markdown(
    """
    Enter any number less than or equal to 'the total number of Graphs Generated'.
    
    """)
    
    inp3 = gr.Number(label="Specify the graph number you want to visualize",value=1)
    btn2 = gr.Button("Visualise Connectivity Graphs",variant='primary')
    out2 = gr.Plot()
    btn2.click(fn=visualise, inputs=inp3, outputs=out2)

In [88]:
demo = gr.TabbedInterface([demo1, demo2], ["Visualize frequency content", "Visualize functional connectivity"])

if __name__ == "__main__":
    demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7912/
Running on public URL: https://31565.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


In [ ]:
# Author: Teon Brooks <teon.brooks@gmail.com>
#
# License: BSD (3-clause)
from mne.datasets import sample
from mne.io import read_raw_fif

from mne_realtime import LSLClient, MockLSLStream

def mock_stream():

    freq_image_dynamic = ''

    # this is the host id that identifies your stream on LSL
    host = 'mne_stream'
    # this is the max wait time in seconds until client connection
    wait_max = 5


    # Load a file to stream raw data
    data_path = sample.data_path()
    raw_fname = data_path  / 'MEG' / 'sample' / 'sample_audvis_filt-0-40_raw.fif'
    raw = read_raw_fif(raw_fname).crop(0, 30).load_data().pick('eeg')

    # For this example, let's use the mock LSL stream.
    _, ax = plt.subplots(1)
    n_epochs = 5

    # main function is necessary here to enable script as own program
    # in such way a child process can be started (primarily for Windows)
    if __name__ == '__main__':
        with MockLSLStream(host, raw, 'eeg'):
            with LSLClient(info=raw.info, host=host, wait_max=wait_max) as client:
                client_info = client.get_measurement_info()
                sfreq = int(client_info['sfreq'])

                # let's observe ten seconds of data
                for ii in range(n_epochs):
                    print('Got epoch %d/%d' % (ii + 1, n_epochs))
                    plt.cla()
                    epoch = client.get_data_as_epoch(n_samples=sfreq)
                    epoch.average().plot(axes=ax).savefig('freq_img_'+str(ii)+'.png')
                    freq_image_dynamic = 'freq_img_'+str(ii)+'.png'
                    plt.pause(3.)
                #plt.draw()
    #print('Streams closed')

In [87]:
with gr.Blocks() as demo6:
    button = gr.Button("Stream")
    stream_visual = gr.Image(value='neuroinformatics_logo.png')
    
    def stream():
        i=0
        path='freq_img_'+str(i)+'.png'
        print(path)
        return gr.update(value=path)
    def stream2(i):
        i=math.trunc(i)
        #if i<=4:
        path='freq_img_'+str(i)+'.png'
        return path
        
    button.click(fn=stream, inputs=None, outputs=stream_visual)
    nbr = gr.Number(value=1, visible=False)
    for i in range(5):
        nbr.update(value=i)
        stream_visual.change(stream2,nbr,stream_visual)
        print(i)

demo6.launch()

0
1
2
3
4
Running on local URL:  http://127.0.0.1:7911/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x29b91ab80>, 'http://127.0.0.1:7911/', None)

freq_img_0.png
